# Semestrální práce KMA/NA

Jan Půlpán

## Úkoly 4.-6.

In [3]:
import numpy as np
import numpy.linalg as la
from rogues import lauchli # Python and Numpy port of Prof. Nicholas Higham's matlab test matrices
import pandas as pd
import time

np.set_printoptions(precision=4, suppress=True)

### 4. Příprava kódů na výpočet QR rozkladu.

Implementujeme algoritmy pro QR rozklad obecné obdélníkové matice $A \in \mathbb{R}^{n \times m}$ tak, že platí $$A = QR,$$ kde $Q \in \mathbb{R}^{n \times n}$ je ortogonální matice a $R \in \mathbb{R}^{n \times m}$ je horní trojúhelníková matice. Implementované algoritmy jsou:
- QR rozklad pomocí Householderových reflexí,
- (iterovaný) klasický gram-schmidtův algoritmus,
- (iterovaný) modifikovaný gram-schmidtův algoritmus.

In [4]:
# QR rozklad pomocí Householderových reflexí
def qr_householder(A):
    
    n, m = A.shape
    o = np.min([n,m])
    if n > m: 
        o += 1
    R = np.copy(A)
    Q = np.eye(n)

    for i in range(o-1):
        v = np.copy(R[i:,i])
        
        v[0] = v[0] + np.sign(v[0]) * la.norm(v)
        v = (v / la.norm(v)) #.reshape(-1,1)

        Q_i = np.eye(n)
        Q_i[i:,i:] = np.eye(n-i) - 2 * np.outer(v,v) #v @ v.T

        Q = Q_i @ Q
        R = Q_i @ R
    
    return Q.T, R

In [5]:
# (iterovaný) klasický Gram-Schmidtův algoritmus
def qr_cgs(A, it=1):
    m, n = A.shape
    s = np.min([m,n])
    
    R = np.zeros((s,n))
    r = np.zeros((s,n))
    Q = np.zeros((m,s))

    for i in range(n):
        z = A[:,i]
        
        for k in range(it):
            for j in range(np.min([m,i])):
                r[j, i] = z @ Q[:,j]

            for j in range(np.min([m,i])):
                z = z - Q[:,j] * r[j,i]
                R[j, i] += r[j, i]

        
        if i < m:
            R[i,i] = la.norm(z)
            Q[:, i] = z / R[i,i]
        
    return Q, R

In [6]:
# (iterovaný) modifikovaný Gram-Schmidtův algoritmus
def qr_mgs(A, it=1):
    m, n = A.shape
    s = np.min([m,n])
    
    Q = np.zeros((m,s))
    r = np.zeros((s,n))
    R = np.zeros((s,n))

    for i in range(n):
        z = A[:,i]
        
        for k in range(it):
            for j in range(np.min([m,i])):
                r[j, i] = z @ Q[:,j]

            #for j in range(np.min([m,i])):
                z = z - Q[:,j] * r[j,i]
                R[j, i] += r[j, i]

        
        if i < m:
            R[i,i] = la.norm(z)
            Q[:, i] = z / R[i,i]
        
    return Q, R

### 5. Test numerické stability výpočtu QR rozkladu

Numerickou stabilitu implementovaných algoritmů otestujeme na výpočtu QR rozkladu Lauchliho matice velikosti $21 \times 20$ s parametrem $\delta=10^{-7}$. Pro takto vypočtené rozklady vyjádříme ztrátu ortogonality $\Vert I - Q^TQ \Vert_F$ a normu rezidua $\Vert A - QR \Vert$. Výsledky jsou shrnuty v tabulce pod výpočty.

In [7]:
def ortogonality_loss(Q):
    Qsqr = Q.T @ Q
    return la.norm(np.identity(len(Qsqr)) - Qsqr)


def res_norm(A, Q, R):
    return la.norm(A - Q @ R, ord=2)

In [8]:
A = lauchli(20, 1e-7)
results = []

Q, R = qr_householder(A)

results.append(['QR Householder',
                ortogonality_loss(Q),
                res_norm(A, Q, R)])


In [9]:
Q, R = qr_cgs(A)

results.append(['CGS',
                ortogonality_loss(Q),
                res_norm(A, Q, R)])

In [10]:
Q, R = qr_mgs(A)

results.append(['MGS',
                ortogonality_loss(Q),
                res_norm(A, Q, R)])

In [11]:
Q, R = qr_cgs(A, it=2)

results.append(['ICGS',
                ortogonality_loss(Q),
                res_norm(A, Q, R)])

In [12]:
Q, R = qr_mgs(A, it=2)

results.append(['IMGS',
                ortogonality_loss(Q),
                res_norm(A, Q, R)])

In [13]:
Q, R = np.linalg.qr(A)

results.append(['QR Numpy',
                ortogonality_loss(Q),
                res_norm(A, Q, R)])

Jednotlivé algoritmy na výpočet QR rozkladu jsou porovnány pomocí ztráty ortogonality a normy rezidua v následující tabulce. 

Pro normu rezidua a strojovou přesnost $\epsilon$ použitého počítače a datového typu `float64` platí $$\Vert A - QR \Vert \approx \epsilon \Vert A \Vert.$$
Hodnoty normy rezidua vypočtené pro použité algoritmy tento vztah potvrzují (viz. poslední sloupec tabulky) a jsou přibližně rovny hodnotě normy rezidua s využitím strojové přesnosti $\epsilon$ a normy matice $A$.

Ztráta ortogonality $\Vert I - Q^TQ \Vert_F$ ve Frobeniově normě je uvedena v třetím sloupci tabulky. U CGS pro ztrátu ortogonality platí $\kappa^2(A) \epsilon$, pro MGS pak $\kappa(A) \epsilon$. U ostatních algoritmů je ztráta ortogonality úměrná jen počítačové přesnosti $\epsilon$. Tyto vztahy jsou výpočty potvrzeny.

In [14]:
eps = np.finfo(np.float64).eps
print(f'𝜖‖𝐴‖ ≈ {eps * la.norm(A,ord=2)}')
k_A = la.cond(A)
print(f'𝜅2(𝐴)𝜖: {k_A**2 * eps}')
print(f'𝜅(𝐴)𝜖: {k_A * eps}')

df = pd.DataFrame(results, columns = ['Metoda',
                                      'Ztráta ortogonality',
                                      'Norma rezidua'])
df

𝜖‖𝐴‖ ≈ 9.930136612989096e-16
𝜅2(𝐴)𝜖: 0.4440892098500634
𝜅(𝐴)𝜖: 9.9301366129891e-09


,Metoda,Ztráta ortogonality,Norma rezidua
0,QR Householder,3.220291e-15,5.352150e-15
1,CGS,3.088932e-02,7.021667e-16
2,MGS,3.170847e-09,1.922963e-16
3,ICGS,6.960196e-16,7.021667e-16
4,IMGS,7.259458e-16,7.021667e-16
5,QR Numpy,1.438665e-15,1.554312e-15


### 6. Test rychlosti a přesnosti algoritmů

Na dostatečné velké náhodné matici porovnáme výsledky všech implementovaných algoritmů a QR rozkladu pomocí funkce `numpy.linalg.qr`. Sledujeme tyto charakteristiky 
- čas výpočtu,
- přesnost výpočtu měřenou ztrátou ortogonality a normou rezidua.

Výsledky jsou opět shrnuty v tabulce.

In [15]:
A = np.random.uniform(-100,100,(1200,400))
results = []

In [16]:
t0 = time.time()
Q, R = qr_householder(A)
t1 = time.time()

results.append(['QR Householder',
                t1-t0, 
                ortogonality_loss(Q),
                res_norm(A, Q, R)])

In [17]:
t0 = time.time()
Q, R = qr_cgs(A)
t1 = time.time()

results.append(['CGS',
                t1-t0, 
                ortogonality_loss(Q),
                res_norm(A, Q, R)])

In [18]:
t0 = time.time()
Q, R = qr_mgs(A)
t1 = time.time()

results.append(['MGS',
                t1-t0, 
                ortogonality_loss(Q),
                res_norm(A, Q, R)])

In [19]:
t0 = time.time()
Q, R = qr_cgs(A, it=2)
t1 = time.time()

results.append(['ICGS',
                t1-t0, 
                ortogonality_loss(Q),
                res_norm(A, Q, R)])

In [20]:
t0 = time.time()
Q, R = qr_mgs(A, it=2)
t1 = time.time()
results.append(['IMGS',
                t1-t0, 
                ortogonality_loss(Q),
                res_norm(A, Q, R)])

In [21]:
t0 = time.time()
Q, R = la.qr(A)
t1 = time.time()

results.append(['QR Numpy',
                t1-t0, 
                ortogonality_loss(Q),
                res_norm(A, Q, R)])

In [22]:
eps = np.finfo(np.float64).eps
print(f'𝜖‖𝐴‖ ≈ {eps * la.norm(A,ord=2)}')
k_A = la.cond(A)
print(f'𝜅2(𝐴)𝜖: {k_A**2 * eps}')
print(f'𝜅(𝐴)𝜖: {k_A * eps}')
df = pd.DataFrame(results, columns = ['Metoda',
                                      'Čas (s)',
                                      'Ztráta ortogonality',
                                      'Norma rezidua'])
df

𝜖‖𝐴‖ ≈ 6.931024015705656e-13
𝜅2(𝐴)𝜖: 2.8911635831939313e-15
𝜅(𝐴)𝜖: 8.012286038353439e-16


,Metoda,Čas (s),Ztráta ortogonality,Norma rezidua
0,QR Householder,14.149284,4.805411e-13,3.633230e-11
1,CGS,1.190908,1.593082e-14,2.327689e-12
2,MGS,0.719419,1.308928e-14,2.335811e-12
3,ICGS,2.428116,1.072286e-14,2.901700e-12
4,IMGS,1.404417,9.725456e-15,2.797087e-12
5,QR Numpy,0.028009,1.186650e-14,2.936302e-12


Ztráta ortogonality i norma rezidua je srovnatelná pro všechny implementované metody. Na rozdíl od Luchliho matice má náhodná matice $A$ mnohem nižší číslo podmíněnosti a to tedy neovlivní tolik ztrátu ortogonality.

Obecně potřebuje QR rozklad pomocí Householderových reflexi méně operací než CGS nebo MGS. Mělo by tedy platit, že bude proveden i v kratším čase. Toto se ale výpočty nepotvrzuje. Je to způsobeno jak větší výpočetní náročností těchto operací, tak hlavně neoptimalizovaným algoritmem. Z tabulky vidíme, že pro matici $A \in \mathbb{C}^{600 \times 300}$, tedy $n > m$ je čas pro výpočet QR rozkladu pomocí Householderových reflexí výrazně delší, než v případě CGS/MGS nebo i jejich iterovaných variant. Pro čtvercové matice nebo matice kde $n < m$ může být implementovaný QR rozklad pomocí Householderových reflexí rychlejší relativně k ostatním metodám, vždy bude ale nejpomalejší absolutně. 

CGS a MGS k výpočtu potřebují stejný počet operací, délka běhu těchto algoritmů je tedy porovnatelná. Potvrzuje se také teoretický předpoklad, že iterované verze CGS a MGS vyžadují přibližně 2x delší čas běhu.

Nejkratší čast výpočtu má interní funkce `numpy.linalg.qr`, která je ovšem interfacem na Fortran90 knihovnu LAPACK, která je kompilovaná a její čas běhu nelze tedy moc porovnávat s interpretovaným kódem.

In [20]:
eps = np.finfo(np.float64).eps
print(f'𝜖‖𝐴‖ ≈ {eps * la.norm(A,ord=2)}')
k_A = la.cond(A)
print(f'𝜅2(𝐴)𝜖: {k_A**2 * eps}')
print(f'𝜅(𝐴)𝜖: {k_A * eps}')
df = pd.DataFrame(results, columns = ['Metoda',
                                      'Čas (s)',
                                      'Ztráta ortogonality',
                                      'Norma rezidua'])
df

𝜖‖𝐴‖ ≈ 6.977540197617991e-13
𝜅2(𝐴)𝜖: 3.1036876896898398e-15
𝜅(𝐴)𝜖: 8.301548692068629e-16


,Metoda,Čas (s),Ztráta ortogonality,Norma rezidua
0,QR Householder,48.911872,4.682719e-13,3.557629e-11
1,CGS,2.413456,1.599495e-14,2.339853e-12
2,MGS,1.258907,1.304379e-14,2.346816e-12
3,ICGS,3.099307,1.065889e-14,2.901965e-12
4,IMGS,2.210634,9.806418e-15,2.795871e-12
5,QR Numpy,0.059884,1.208366e-14,2.978237e-12
